# Omar Elsayed 43-121

In [1]:
import pandas as pd
df = pd.read_csv ('fraud_detection.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [2]:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import load_boston
    from sklearn.metrics import mean_squared_error
    from sklearn.linear_model import LogisticRegression

In [3]:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import load_boston
    from sklearn.metrics import mean_squared_error
    from sklearn.linear_model import LogisticRegression
#drop nameOrig and nameDest becare ause they are unique and are string so most probably they are of no effect to 
    df_X=df.drop('isFraud',axis=1)
    df_X.drop('nameOrig',axis=1,inplace=True)
    df_X.drop('nameDest',axis=1,inplace=True)
    df_Y=df['isFraud']

    from sklearn.preprocessing import OrdinalEncoder

    ord_enc = OrdinalEncoder()
    df_X["type"] = ord_enc.fit_transform(df_X[["type"]])
#degree 1 and 2 only and max iterations 50 only to allow the code to run because more than that the code terminates due to lack of memory.
    poly_reg_rmsearray=[]
    for i in range(1,3,1):
      poly = PolynomialFeatures(degree= i, include_bias=False)
      poly_features = poly.fit_transform(df_X)
      X_train, X_test, y_train, y_test = train_test_split(poly_features, df_Y, test_size=0.4, random_state=1)
      X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
      poly_reg_model = LogisticRegression( max_iter=50)
      poly_reg_model.fit(X_train, y_train)
      poly_reg_y_predicted = poly_reg_model.predict(X_val)
      poly_reg_rmse = np.sqrt(mean_squared_error(y_val, poly_reg_y_predicted))
      poly_reg_rmsearray.append(poly_reg_rmse)

C:\Users\LAPTOP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LAPTOP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_

In [4]:
print(poly_reg_rmsearray)


[0.03677540330372647, 0.036258929630713754]


In [5]:
#validate for degree 2 since it gave the least rmse 
poly = PolynomialFeatures(degree= 2, include_bias=False)
poly_features = poly.fit_transform(df_X)
X_train, X_test, y_train, y_test = train_test_split(poly_features, df_Y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
poly_reg_model = LogisticRegression(max_iter=50)
poly_reg_model.fit(X_train, y_train)
poly_reg_y_predicted = poly_reg_model.predict(X_test)
poly_reg_rmse = np.sqrt(mean_squared_error(y_test, poly_reg_y_predicted))
print(poly_reg_rmse)

C:\Users\LAPTOP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.03553652418932332


## Stratified K-fold

In [6]:
from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
lst_accu_stratified = []
  
for train_index, test_index in skf.split(poly_features, df_Y):
    x_train_fold, x_test_fold = poly_features[train_index], poly_features[test_index]
    y_train_fold, y_test_fold = df_Y[train_index], df_Y[test_index]
    poly_reg_model = LogisticRegression(max_iter=50)
    poly_reg_model.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(poly_reg_model.score(x_test_fold, y_test_fold))
  
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

C:\Users\LAPTOP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LAPTOP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

List of possible accuracy: [0.9987090227896612, 0.9987094936849118, 0.9987090221809604]

Maximum Accuracy That can be obtained from this model is: 99.87094936849118 %

Minimum Accuracy: 99.87090221809603 %

Overall Accuracy: 99.87091795518445 %

Standard Deviation is: 2.7204738672649894e-07


## Regularization

In [10]:
from sklearn.linear_model import LogisticRegressionCV 
alphaarray=[0.1,0.01,1]
result=[]

X_train, X_test, y_train, y_test = train_test_split(poly_features, df_Y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
  
poly_reg_model = LogisticRegression(penalty='l1', C=1 ,solver='saga', max_iter=2)
poly_reg_model.fit(X_train, y_train)
poly_reg_y_predicted = poly_reg_model.predict(X_test)
poly_reg_rmse = np.sqrt(mean_squared_error(y_test, poly_reg_y_predicted))
result.append(poly_reg_rmse)

C:\Users\LAPTOP\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [11]:
print(result)

[0.03553652418932332]
